In [1]:
import os
import warnings
import re
from itertools import chain

import scipy
import scipy.io
import pandas

import cobra

try:
    from read_excel import read_excel
except ImportError:
    !wget https://raw.githubusercontent.com/opencobra/m_model_collection/master/read_excel.py
    from read_excel import read_excel

In [2]:
def open_exchanges(model, amount=10):
    for reaction in model.reactions:
        if len(reaction.metabolites) == 1:
            # Ensure we are not creating any new sinks
            if reaction.metabolites.values()[0] > 0:
                reaction.upper_bound = max(reaction.upper_bound, amount)
            else:
                reaction.lower_bound = min(reaction.lower_bound, -amount)

def add_exchanges(model, extracellular_suffix="[e]", uptake_amount=10):
    for metabolite in model.metabolites:
        if str(metabolite).endswith(extracellular_suffix):
            if len(metabolite.reactions) == 0:
                print "no reactions for " + metabolite.id
                continue
            if min(len(i.metabolites) for i in metabolite.reactions) > 1:
                EX_reaction = cobra.Reaction("EX_" + metabolite.id)
                EX_reaction.add_metabolites({metabolite: 1})
                m.add_reaction(EX_reaction)
                EX_reaction.upper_bound = uptake_amount
                EX_reaction.lower_bound = -uptake_amount

In [3]:
models = cobra.DictList()

In [4]:
m = read_excel("iMH551.xls", rxn_sheet_name="GPR Annotation", rxn_sheet_header=4, rxn_id_key="auto", rxn_str_key="REACTION",
           rxn_name_key="ENZYME", rxn_skip_rows=[625, 782, 787], verbose=False)
add_exchanges(m, "(extracellular)")
models.append(m)

In [5]:
m = read_excel("iLL672.xls",
               rxn_id_key="auto", met_sheet_name="Appendix 3 iLL672 metabolites",\
               rxn_str_key="REACTION", verbose=False,
               rxn_sheet_name='Appendix 3 iLL672 reactions')
m.reactions[-1].objective_coefficient = 1
add_exchanges(m, "xt")
models.append(m)

In [6]:
m = read_excel("iOA584.xls", verbose=False)
add_exchanges(m, "xt")
models.append(m)

In [7]:
m = read_excel("iBT721.xls", rxn_sheet_name="reaction info", rxn_skip_rows=[745], verbose=False)
m.reactions[-2].objective_coefficient = 1
models.append(m)

In [8]:
m = read_excel("iRS605.xls",
               met_id_key="METABOLITE ABBREVIATION", met_name_key="METABOLITE NAME", verbose=False)
add_exchanges(m)
models.append(m)

In [9]:
m = read_excel("iGT196.xls", verbose=False)
add_exchanges(m)
models.append(m)

In [10]:
m = read_excel("iGB555.xls", rxn_sheet_header=5,
               rxn_id_key="auto", rxn_str_key="Reaction",
               verbose=False, rxn_reversible_arrow="=", rxn_fwd_arrow=">", 
               rxn_sheet_name="simplified model")
models.append(m)

In [11]:
m = read_excel("iKK446.xls", rxn_sheet_name="List of Reactions", rxn_sheet_header=1,
               rxn_id_key="Gene name", rxn_str_key="Reactions", rxn_gpr_key="ORF", verbose=False)
add_exchanges(m, "xt")
models.append(m)

In [12]:
m = read_excel("iYS277.xls", rxn_id_key="auto", rxn_str_key="Unnamed: 3", rxn_gpr_key="gene name", verbose=False)
for met in m.metabolites:
    if met.id.startswith("[e]"):
        met.id = met.id[3:] + "[e]"
m.repair()
add_exchanges(m)

no reactions for ACET[e]
no reactions for OXYGEN-MOLECULE[e]
no reactions for SO3[e]
no reactions for AMMONIA[e]


In [13]:
for i in os.listdir("."):
    if not i.endswith(".xml"):
        continue
    model = cobra.io.read_legacy_sbml(i)
    model.id = i[:-4]
    models.append(model)

## Autodetect objectives

In [14]:
biomass_re = re.compile("biomass", re.IGNORECASE)
curated_objectives = {"iOA584": "rxn1387", "iIN800": "GROWTH", "iGT196": "VGRO", "iMH551": "R0227"}

In [15]:
for m in models:
    if len(m.reactions.query(lambda x: x > 0, "objective_coefficient")):
        continue
    if m.id in curated_objectives:
        m.change_objective(curated_objectives[m.id])
        continue
    
    # look for reactions with "biomass" in the id or name
    possible_objectives = m.reactions.query(biomass_re)
    if len(possible_objectives) == 0:
        possible_objectives = m.reactions.query(biomass_re, "name")
    
    # In some cases, a biomass "metabolite" is produced, whose production
    # should be the objective function.
    possible_biomass_metabolites = m.metabolites.query(biomass_re)
    if len(possible_biomass_metabolites) == 0:
        possible_biomass_metabolites = m.metabolites.query(biomass_re, "name")

    if len(possible_biomass_metabolites) > 0:
        biomass_met = possible_biomass_metabolites[0]
        r = cobra.Reaction("added_biomass_sink")
        r.objective_coefficient = 1
        r.add_metabolites({biomass_met: -1})
        m.add_reaction(r)
        print ("autodetected biomass metabolite '%s' for model '%s'" %
              (biomass_met.id, m.id))

    elif len(possible_objectives) > 0:
        print("autodetected objective reaction '%s' for model '%s'" %
              (possible_objectives[0].id, m.id))
        m.change_objective(possible_objectives[0])

    
    else:
        print("no objective found for " + m.id)
        models.remove(m.id)

# Ensure the biomass objective flux is unconstrained
for reaction in m.reactions.query(lambda x: x > 0, "objective_coefficient"):
    reaction.lower_bound = min(reaction.lower_bound, 0)
    reaction.upper_bound = max(reaction.upper_bound, 1000)

autodetected objective reaction 'FT_Biomx_DM' for model 'iRS605'
autodetected biomass metabolite 'BIOMASS' for model 'iGB555'
autodetected biomass metabolite 'BIOMASS' for model 'iKK446'
autodetected biomass metabolite 'BIOMASS' for model 'iHD666'
autodetected biomass metabolite 'Biomass_c' for model 'PlasmoNet'
no objective found for iMH805


cobra/core/Model.py:316 UserWarning: Deprecated. Use the objective setter (Model.objective = ...) instead


## Solve

In [16]:
results = {}
for m in models:
    # solve this model with all the solvers
    solutions = {solver: m.optimize(solver=solver)
                 for solver in cobra.solvers.solver_dict}
    solutions["cglpk_exact"] = m.optimize(solver="cglpk", exact=True)
    # store the objective value and errors
    results[m.id] = {k: v.f for k, v in solutions.iteritems()}

results = pandas.DataFrame.from_dict(results, orient='index')
results

,mosek,esolver,cglpk,cplex,gurobi,cglpk_exact,coin,glpk
PlasmoNet,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,-1.140600e-13,0.000000e+00
iBT721,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,0.000000e+00,0.000000e+00
iGB555,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,0.000000e+00,0.000000e+00
iGT196,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,7.506453e-08,0.000000e+00
iHD666,1.894781e-10,0,1.790139e-10,0.000000e+00,-0,0,0.000000e+00,0.000000e+00
iIN800,0.000000e+00,0,4.144976e-14,1.136868e-13,-0,0,2.116824e-11,3.889961e-15
iKK446,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,5.872481e-10,6.705318e-29
iLL672,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,0.000000e+00,0.000000e+00
iMH551,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,0.000000e+00,0.000000e+00
iOA584,0.000000e+00,0,0.000000e+00,0.000000e+00,-0,0,5.036499e-09,0.000000e+00


In [17]:
abs(results).max().max()

7.5064531298271003e-08